# Intro to ML Capstone Project 
* Professor: Lerrel Pinto
* Made by Sangwon Baek
* December 10th 2022
* Kaggle Site URL:
https://www.kaggle.com/competitions/csci-ua-473-intro-to-machine-learning-fall22/overview
* Batchsize 32 was used
* ResNet 152 / LR : 0.001 (1e-3)
* Applied normalization for RGB by calculating them / for depth(divided by 1000 first) used CV2 normalizer minmax
* Test/Valid split (8:2), implemented earlystopping (patience=5, stepLR on every epoch with Gamma 0.8 step 1)
* Did (targets) ground truth * 100 before passing to the loss function (meters to mm) then divided by 100 before submission

In [1]:
import os 
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torchvision import models
from torchvision.models import resnet50, ResNet50_Weights, resnet18, ResNet18_Weights, resnet152, ResNet152_Weights

from torchsummary import summary

import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import cv2
import pickle as pkl 

### Preprocessing and loading dataset

In [2]:
class LazyLoadDataset(Dataset):
    def __init__(self, path, train=True, transform=None):
        self.transform = transform
        self.train = train
        self.transform_0 = None
        self.transform_1 = None
        self.transform_2 = None
        
        if self.transform is not None:
            self.transform_0 = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(240),
            transforms.CenterCrop(224),
            transforms.Normalize([0.4352, 0.4170, 0.3960], [0.1992, 0.1987, 0.2111])
            ])
            self.transform_1 = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(240),
            transforms.CenterCrop(224),
            transforms.Normalize([0.5008, 0.4879, 0.4697], [0.2276, 0.2252, 0.2417])
            ])
            self.transform_2 = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(240),
            transforms.CenterCrop(224),
            transforms.Normalize([0.5193, 0.4820, 0.4412], [0.2293, 0.2288, 0.2465])
            ])
        path = path + ("train/" if train else "test/")
        
        self.pathX = path+"X/"
        self.pathY = path+"Y/"
        
        self.data = os.listdir(self.pathX)
    
    def __getitem__(self, idx):
        f = self.data[idx]
        
        #Read rgb images
        img0 = cv2.imread(self.pathX + f + '/rgb/0.png')
        img1 = cv2.imread(self.pathX + f + '/rgb/1.png')
        img2 = cv2.imread(self.pathX + f + '/rgb/2.png')
        
        #read depth images
        depth = np.load(self.pathX + f + '/depth.npy')        
        depth = depth/1000
        
        #Convert RGB and depth images to tensor
        if self.transform is not None:
            img0 = self.transform_0(img0)
            img1 = self.transform_1(img1)
            img2 = self.transform_2(img2)
        
        #Perform transformation on Depth image
        depth = cv2.normalize(depth, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)  
            
        #read field ID & Y
        field_id = pkl.load(open(self.pathX + f + '/field_id.pkl', 'rb'))
        
        if self.train==True:
            Y = np.load(self.pathY + f + '.npy')
            return (img0, img1, img2, depth, field_id), Y
        if self.train==False:
            return (img0, img1, img2, depth, field_id)
        
    def __len__(self):
        return len(self.data)

### Explore data shape

In [3]:
#Transform to tensor
transform = transforms.ToTensor()

#Lazy Load the dataset
dataset = LazyLoadDataset('../lazydata/',train=True, transform=transform)


In [4]:
#Define train/validation size (8:2)
train_size = int(len(dataset) * 0.8)
validation_size = len(dataset)-train_size

#Randomly split dataset into train and validation dataset with specified size above
train_dataset, validation_dataset = random_split(dataset, [train_size, validation_size])

#Create train/validation dataloader with batch_size of 64, 32 respectively
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=32, shuffle=True)


In [5]:
print("Train set size: {} \nTrain Loader size: {}".format(len(train_dataset),len(train_dataloader)))
print("Validation set size: {} \nValidation Loader size: {}".format(len(validation_dataset),len(validation_dataloader)))

Train set size: 2716 
Train Loader size: 85
Validation set size: 680 
Validation Loader size: 22


In [6]:
for i, ((img0, img1, img2, depth, field_id), labels) in enumerate(train_dataloader):
    print(i)
    # print(depth[0])
    print("img0 shape{}".format(img0.size()))
    print("img1 shape{}".format(img1.size()))
    print("img2 shape{}".format(img2.size()))
    print("depth shape{}".format(depth.shape))
    print("field id {}".format(field_id))
    print("labels size {}".format(labels.size()))
    break

0
img0 shapetorch.Size([32, 3, 224, 224])
img1 shapetorch.Size([32, 3, 224, 224])
img2 shapetorch.Size([32, 3, 224, 224])
depth shapetorch.Size([32, 3, 224, 224])
field id ('3617', '3175', '2547', '2973', '2878', '1370', '1478', '1040', '4005', '1758', '251', '1856', '2774', '160', '3816', '1977', '1231', '698', '3339', '516', '892', '1049', '3471', '2712', '2818', '2187', '3036', '502', '1943', '1749', '710', '1781')
labels size torch.Size([32, 12])


### Set device to cuda & Define parameters

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 12
lr = 1e-3
epochs = 50

### Train 1st img 

In [8]:
def train_1(epoch, model, optimizer, scheduler):
    # Early stopping Parameters
    Best_Val_loss = None
    patience = 5
    trigger_times = 0
    for epoch in range(0, epochs):
        model.train()
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(train_dataloader): 
            #Stack all three RGB 
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:]),1)
            data = img_all
            data = data.to(device)

            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            #Run necessary train implementations
            output = model(data)
            
            #Zero gradients
            optimizer.zero_grad()

            #forward and backward propagation
            loss_function = nn.MSELoss()
            #Root Mean Squared Error (RMSE) 
            train_loss = torch.sqrt(loss_function(output.float(), target.float()))
            train_loss.backward()
            optimizer.step()

            #Show progress
            if batch_idx % 28 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dataloader.dataset),
                    100. * batch_idx / len(train_dataloader), train_loss.item()))
   
        #To help model improving apply scheduler to reduce learning rate
        scheduler.step()
        curr_lr = scheduler.get_last_lr()
        print("current lr{}".format(curr_lr))     
        
        #Early Stopping
        Curr_Val_loss = validation_1(model)
        print('Valid Epoch: {} Current Validation Loss: {:.6F}'.format(epoch, Curr_Val_loss))
        
        if Curr_Val_loss < 0.242:
            print ("Best model obtained! Set Patience = 2")
            patience = 2 
        
        if Best_Val_loss is None:
            Best_Val_loss = Curr_Val_loss - 0.0001
        #Model didn't improve so add trigger time
        if Curr_Val_loss > Best_Val_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        #Reset to trigger_time 0 and update best validation loss
        else:
            trigger_times = 0
            Best_Val_loss = Curr_Val_loss
            print('trigger times: 0 Best_Val_loss: {:.6F}'.format(Best_Val_loss)) 

    return model

In [9]:
def validation_1(model):
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(validation_dataloader):
            #Stack all three RGB and depth images
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:]),1)
            data = img_all
            data = data.to(device)
            
            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            output = model(data)
            loss_function = nn.MSELoss()
            loss = torch.sqrt(loss_function(output.float(), target.float()))
            loss_total += loss.item()
    return loss_total / len(validation_dataloader)

In [10]:
model_1 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
model_1.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_1.fc = nn.Linear(2048, num_classes)
model_1 = model_1.cuda()
optimizer = torch.optim.Adam(model_1.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
model_1 = train_1(epochs, model_1, optimizer, scheduler)

Train Epoch: 0 [0/2716 (0%)]	Loss: 6.830080
Train Epoch: 0 [896/2716 (33%)]	Loss: 0.993739
Train Epoch: 0 [1792/2716 (66%)]	Loss: 0.879711
Train Epoch: 0 [2352/2716 (99%)]	Loss: 0.763808
current lr[0.0008]
Valid Epoch: 0 Current Validation Loss: 0.797618
trigger times: 1
Train Epoch: 1 [0/2716 (0%)]	Loss: 0.694618
Train Epoch: 1 [896/2716 (33%)]	Loss: 0.656592
Train Epoch: 1 [1792/2716 (66%)]	Loss: 0.621730
Train Epoch: 1 [2352/2716 (99%)]	Loss: 0.427999
current lr[0.00064]
Valid Epoch: 1 Current Validation Loss: 0.446688
trigger times: 0 Best_Val_loss: 0.446688
Train Epoch: 2 [0/2716 (0%)]	Loss: 0.441339
Train Epoch: 2 [896/2716 (33%)]	Loss: 0.510876
Train Epoch: 2 [1792/2716 (66%)]	Loss: 0.417094
Train Epoch: 2 [2352/2716 (99%)]	Loss: 0.384095
current lr[0.0005120000000000001]
Valid Epoch: 2 Current Validation Loss: 0.445030
trigger times: 0 Best_Val_loss: 0.445030
Train Epoch: 3 [0/2716 (0%)]	Loss: 0.526889
Train Epoch: 3 [896/2716 (33%)]	Loss: 0.286180
Train Epoch: 3 [1792/2716 (66

In [11]:
#Save my CNN model
torch.save(model_1.state_dict(), "../model/model_img1.pt")

### Train 2nd image

In [8]:
def train_2(epoch, model, optimizer, scheduler):
    # Early stopping Parameters
    Best_Val_loss = None
    patience = 7
    trigger_times = 0
    for epoch in range(0, epochs):
        model.train()
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(train_dataloader): 
            #Stack all three RGB 
            img_all = torch.stack((img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:]),1)
            
            data = img_all
            data = data.to(device)

            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            #Run necessary train implementations
            output = model(data)
            
            #Zero gradients
            optimizer.zero_grad()

            #forward and backward propagation
            loss_function = nn.MSELoss()
            #Root Mean Squared Error (RMSE) 
            train_loss = torch.sqrt(loss_function(output.float(), target.float()))
            train_loss.backward()
            optimizer.step()

            #Show progress
            if batch_idx % 28 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dataloader.dataset),
                    100. * batch_idx / len(train_dataloader), train_loss.item()))
   
        #To help model improving apply scheduler to reduce learning rate
        scheduler.step()
        curr_lr = scheduler.get_last_lr()
        print("current lr{}".format(curr_lr))     
        
        #Early Stopping
        Curr_Val_loss = validation_2(model)
        print('Valid Epoch: {} Current Validation Loss: {:.6F}'.format(epoch, Curr_Val_loss))
        
        if Curr_Val_loss < 0.242:
            print ("Best model obtained! Set Patience = 2")
            patience = 2 
        
        if Best_Val_loss is None:
            Best_Val_loss = Curr_Val_loss - 0.0001
        #Model didn't improve so add trigger time
        if Curr_Val_loss > Best_Val_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        #Reset to trigger_time 0 and update best validation loss
        else:
            trigger_times = 0
            Best_Val_loss = Curr_Val_loss
            print('trigger times: 0 Best_Val_loss: {:.6F}'.format(Best_Val_loss)) 

    return model

In [9]:
def validation_2(model):
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(validation_dataloader):
            #Stack all three RGB and depth images
            img_all = torch.stack((img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:]),1)

            data = img_all
            data = data.to(device)
            
            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            output = model(data)
            loss_function = nn.MSELoss()
            loss = torch.sqrt(loss_function(output.float(), target.float()))
            loss_total += loss.item()
    return loss_total / len(validation_dataloader)

In [10]:
model_2 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
model_2.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_2.fc = nn.Linear(2048, num_classes)
model_2 = model_2.cuda()
optimizer = torch.optim.Adam(model_2.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
model_2 = train_2(epochs, model_2, optimizer, scheduler)

Train Epoch: 0 [0/2716 (0%)]	Loss: 6.739582
Train Epoch: 0 [896/2716 (33%)]	Loss: 1.058002
Train Epoch: 0 [1792/2716 (66%)]	Loss: 0.734152
Train Epoch: 0 [2352/2716 (99%)]	Loss: 0.697222
current lr[0.0008]
Valid Epoch: 0 Current Validation Loss: 0.867703
trigger times: 1
Train Epoch: 1 [0/2716 (0%)]	Loss: 0.659827
Train Epoch: 1 [896/2716 (33%)]	Loss: 0.577007
Train Epoch: 1 [1792/2716 (66%)]	Loss: 0.598242
Train Epoch: 1 [2352/2716 (99%)]	Loss: 0.732250
current lr[0.00064]
Valid Epoch: 1 Current Validation Loss: 0.665867
trigger times: 0 Best_Val_loss: 0.665867
Train Epoch: 2 [0/2716 (0%)]	Loss: 0.740123
Train Epoch: 2 [896/2716 (33%)]	Loss: 0.482011
Train Epoch: 2 [1792/2716 (66%)]	Loss: 0.438794
Train Epoch: 2 [2352/2716 (99%)]	Loss: 0.453768
current lr[0.0005120000000000001]
Valid Epoch: 2 Current Validation Loss: 0.560411
trigger times: 0 Best_Val_loss: 0.560411
Train Epoch: 3 [0/2716 (0%)]	Loss: 0.466776
Train Epoch: 3 [896/2716 (33%)]	Loss: 0.797971
Train Epoch: 3 [1792/2716 (66

In [11]:
#Save my CNN model
torch.save(model_2.state_dict(), "../model/model_img2.pt")

### Train 3rd image

In [12]:
def train_3(epoch, model, optimizer, scheduler):
    # Early stopping Parameters
    Best_Val_loss = None
    patience = 7
    trigger_times = 0
    for epoch in range(0, epochs):
        model.train()
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(train_dataloader): 
            #Stack all three RGB 
            img_all = torch.stack((img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)

            data = img_all
            data = data.to(device)

            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            #Run necessary train implementations
            output = model(data)
            
            #Zero gradients
            optimizer.zero_grad()

            #forward and backward propagation
            loss_function = nn.MSELoss()
            #Root Mean Squared Error (RMSE) 
            train_loss = torch.sqrt(loss_function(output.float(), target.float()))
            train_loss.backward()
            optimizer.step()

            #Show progress
            if batch_idx % 28 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dataloader.dataset),
                    100. * batch_idx / len(train_dataloader), train_loss.item()))
   
        #To help model improving apply scheduler to reduce learning rate
        scheduler.step()
        curr_lr = scheduler.get_last_lr()
        print("current lr{}".format(curr_lr))     
        
        #Early Stopping
        Curr_Val_loss = validation_3(model)
        print('Valid Epoch: {} Current Validation Loss: {:.6F}'.format(epoch, Curr_Val_loss))
        
        if Curr_Val_loss < 0.242:
            print ("Best model obtained! Set Patience = 2")
            patience = 2 
        
        if Best_Val_loss is None:
            Best_Val_loss = Curr_Val_loss - 0.0001
        #Model didn't improve so add trigger time
        if Curr_Val_loss > Best_Val_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        #Reset to trigger_time 0 and update best validation loss
        else:
            trigger_times = 0
            Best_Val_loss = Curr_Val_loss
            print('trigger times: 0 Best_Val_loss: {:.6F}'.format(Best_Val_loss)) 

    return model

In [13]:
def validation_3(model):
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(validation_dataloader):
            #Stack all three RGB and depth images
            img_all = torch.stack((img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)

            data = img_all
            data = data.to(device)
            
            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            output = model(data)
            loss_function = nn.MSELoss()
            loss = torch.sqrt(loss_function(output.float(), target.float()))
            loss_total += loss.item()
    return loss_total / len(validation_dataloader)

In [14]:
model_3 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
model_3.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_3.fc = nn.Linear(2048, num_classes)
model_3 = model_3.cuda()
optimizer = torch.optim.Adam(model_3.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
model_3 = train_3(epochs, model_3, optimizer, scheduler)

Train Epoch: 0 [0/2716 (0%)]	Loss: 6.736082
Train Epoch: 0 [896/2716 (33%)]	Loss: 1.150905
Train Epoch: 0 [1792/2716 (66%)]	Loss: 0.839449
Train Epoch: 0 [2352/2716 (99%)]	Loss: 0.713560
current lr[0.0008]
Valid Epoch: 0 Current Validation Loss: 0.770117
trigger times: 1
Train Epoch: 1 [0/2716 (0%)]	Loss: 0.535622
Train Epoch: 1 [896/2716 (33%)]	Loss: 0.800533
Train Epoch: 1 [1792/2716 (66%)]	Loss: 0.482514
Train Epoch: 1 [2352/2716 (99%)]	Loss: 0.440382
current lr[0.00064]
Valid Epoch: 1 Current Validation Loss: 0.639223
trigger times: 0 Best_Val_loss: 0.639223
Train Epoch: 2 [0/2716 (0%)]	Loss: 0.510787
Train Epoch: 2 [896/2716 (33%)]	Loss: 0.382575
Train Epoch: 2 [1792/2716 (66%)]	Loss: 0.387156
Train Epoch: 2 [2352/2716 (99%)]	Loss: 0.461743
current lr[0.0005120000000000001]
Valid Epoch: 2 Current Validation Loss: 0.536974
trigger times: 0 Best_Val_loss: 0.536974
Train Epoch: 3 [0/2716 (0%)]	Loss: 0.503095
Train Epoch: 3 [896/2716 (33%)]	Loss: 0.413035
Train Epoch: 3 [1792/2716 (66

In [15]:
#Save my CNN model
torch.save(model_3.state_dict(), "../model/model_img3.pt")

### Train 1st and 2nd image

In [9]:
def train_12(epoch, model, optimizer, scheduler):
    # Early stopping Parameters
    Best_Val_loss = None
    patience = 5
    trigger_times = 0
    for epoch in range(0, epochs):
        model.train()
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(train_dataloader): 
            #Stack all three RGB 
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:],
                         img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:]),1)
            data = img_all
            data = data.to(device)

            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            #Run necessary train implementations
            output = model(data)
            
            #Zero gradients
            optimizer.zero_grad()

            #forward and backward propagation
            loss_function = nn.MSELoss()
            #Root Mean Squared Error (RMSE) 
            train_loss = torch.sqrt(loss_function(output.float(), target.float()))
            train_loss.backward()
            optimizer.step()

            #Show progress
            if batch_idx % 28 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dataloader.dataset),
                    100. * batch_idx / len(train_dataloader), train_loss.item()))
   
        #To help model improving apply scheduler to reduce learning rate
        scheduler.step()
        curr_lr = scheduler.get_last_lr()
        print("current lr{}".format(curr_lr))     
        
        #Early Stopping
        Curr_Val_loss = validation_12(model)
        print('Valid Epoch: {} Current Validation Loss: {:.6F}'.format(epoch, Curr_Val_loss))
        
        if Curr_Val_loss < 0.242:
            print ("Best model obtained! Set Patience = 2")
            patience = 2 
        
        if Best_Val_loss is None:
            Best_Val_loss = Curr_Val_loss - 0.0001
        #Model didn't improve so add trigger time
        if Curr_Val_loss > Best_Val_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        #Reset to trigger_time 0 and update best validation loss
        else:
            trigger_times = 0
            Best_Val_loss = Curr_Val_loss
            print('trigger times: 0 Best_Val_loss: {:.6F}'.format(Best_Val_loss)) 

    return model

In [10]:
def validation_12(model):
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(validation_dataloader):
            #Stack all three RGB and depth images
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:],
                         img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:]),1)
            data = img_all
            data = data.to(device)
            
            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            output = model(data)
            loss_function = nn.MSELoss()
            loss = torch.sqrt(loss_function(output.float(), target.float()))
            loss_total += loss.item()
    return loss_total / len(validation_dataloader)

In [11]:

model_4 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
model_4.conv1 = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_4.fc = nn.Linear(2048, num_classes)
model_4 = model_4.cuda()
optimizer = torch.optim.Adam(model_4.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
model_4 = train_12(epochs, model_4, optimizer, scheduler)

Train Epoch: 0 [0/2716 (0%)]	Loss: 6.595654
Train Epoch: 0 [896/2716 (33%)]	Loss: 0.954961
Train Epoch: 0 [1792/2716 (66%)]	Loss: 1.124193
Train Epoch: 0 [2352/2716 (99%)]	Loss: 0.564205
current lr[0.0008]
Valid Epoch: 0 Current Validation Loss: 0.737564
trigger times: 1
Train Epoch: 1 [0/2716 (0%)]	Loss: 0.619368
Train Epoch: 1 [896/2716 (33%)]	Loss: 0.500941
Train Epoch: 1 [1792/2716 (66%)]	Loss: 0.623070
Train Epoch: 1 [2352/2716 (99%)]	Loss: 0.554415
current lr[0.00064]
Valid Epoch: 1 Current Validation Loss: 0.608849
trigger times: 0 Best_Val_loss: 0.608849
Train Epoch: 2 [0/2716 (0%)]	Loss: 0.377838
Train Epoch: 2 [896/2716 (33%)]	Loss: 0.346501
Train Epoch: 2 [1792/2716 (66%)]	Loss: 0.421909
Train Epoch: 2 [2352/2716 (99%)]	Loss: 0.378425
current lr[0.0005120000000000001]
Valid Epoch: 2 Current Validation Loss: 0.431401
trigger times: 0 Best_Val_loss: 0.431401
Train Epoch: 3 [0/2716 (0%)]	Loss: 0.424115
Train Epoch: 3 [896/2716 (33%)]	Loss: 0.363512
Train Epoch: 3 [1792/2716 (66

In [12]:
#Save my CNN model
torch.save(model_4.state_dict(), "../model/model_img12.pt")

### Train 1st and 3rd image

In [13]:
def train_13(epoch, model, optimizer, scheduler):
    # Early stopping Parameters
    Best_Val_loss = None
    patience = 5
    trigger_times = 0
    for epoch in range(0, epochs):
        model.train()
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(train_dataloader): 
            #Stack all three RGB 
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:],
                         img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)
            data = img_all
            data = data.to(device)

            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            #Run necessary train implementations
            output = model(data)
            
            #Zero gradients
            optimizer.zero_grad()

            #forward and backward propagation
            loss_function = nn.MSELoss()
            #Root Mean Squared Error (RMSE) 
            train_loss = torch.sqrt(loss_function(output.float(), target.float()))
            train_loss.backward()
            optimizer.step()

            #Show progress
            if batch_idx % 28 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dataloader.dataset),
                    100. * batch_idx / len(train_dataloader), train_loss.item()))
   
        #To help model improving apply scheduler to reduce learning rate
        scheduler.step()
        curr_lr = scheduler.get_last_lr()
        print("current lr{}".format(curr_lr))     
        
        #Early Stopping
        Curr_Val_loss = validation_13(model)
        print('Valid Epoch: {} Current Validation Loss: {:.6F}'.format(epoch, Curr_Val_loss))
        
        if Curr_Val_loss < 0.242:
            print ("Best model obtained! Set Patience = 2")
            patience = 2 
        
        if Best_Val_loss is None:
            Best_Val_loss = Curr_Val_loss - 0.0001
        #Model didn't improve so add trigger time
        if Curr_Val_loss > Best_Val_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        #Reset to trigger_time 0 and update best validation loss
        else:
            trigger_times = 0
            Best_Val_loss = Curr_Val_loss
            print('trigger times: 0 Best_Val_loss: {:.6F}'.format(Best_Val_loss)) 

    return model

In [14]:
def validation_13(model):
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(validation_dataloader):
            #Stack all three RGB and depth images
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:],
                         img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)
            data = img_all
            data = data.to(device)
            
            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            output = model(data)
            loss_function = nn.MSELoss()
            loss = torch.sqrt(loss_function(output.float(), target.float()))
            loss_total += loss.item()
    return loss_total / len(validation_dataloader)

In [15]:
model_5 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
model_5.conv1 = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_5.fc = nn.Linear(2048, num_classes)
model_5 = model_5.cuda()
optimizer = torch.optim.Adam(model_5.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
model_5 = train_13(epochs, model_5, optimizer, scheduler)

Train Epoch: 0 [0/2716 (0%)]	Loss: 6.612373
Train Epoch: 0 [896/2716 (33%)]	Loss: 1.086056
Train Epoch: 0 [1792/2716 (66%)]	Loss: 0.788487
Train Epoch: 0 [2352/2716 (99%)]	Loss: 0.627676
current lr[0.0008]
Valid Epoch: 0 Current Validation Loss: 0.622834
trigger times: 1
Train Epoch: 1 [0/2716 (0%)]	Loss: 0.629399
Train Epoch: 1 [896/2716 (33%)]	Loss: 0.650161
Train Epoch: 1 [1792/2716 (66%)]	Loss: 0.652731
Train Epoch: 1 [2352/2716 (99%)]	Loss: 0.597356
current lr[0.00064]
Valid Epoch: 1 Current Validation Loss: 0.663555
trigger times: 2
Train Epoch: 2 [0/2716 (0%)]	Loss: 0.598870
Train Epoch: 2 [896/2716 (33%)]	Loss: 0.580103
Train Epoch: 2 [1792/2716 (66%)]	Loss: 0.478161
Train Epoch: 2 [2352/2716 (99%)]	Loss: 0.560552
current lr[0.0005120000000000001]
Valid Epoch: 2 Current Validation Loss: 0.656765
trigger times: 3
Train Epoch: 3 [0/2716 (0%)]	Loss: 0.572408
Train Epoch: 3 [896/2716 (33%)]	Loss: 0.345773
Train Epoch: 3 [1792/2716 (66%)]	Loss: 0.509364
Train Epoch: 3 [2352/2716 (99

In [16]:
#Save my CNN model
torch.save(model_5.state_dict(), "../model/model_img13.pt")

### Train 2nd and 3rd image

In [8]:
def train_23(epoch, model, optimizer, scheduler):
    # Early stopping Parameters
    Best_Val_loss = None
    patience = 5
    trigger_times = 0
    for epoch in range(0, epochs):
        model.train()
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(train_dataloader): 
            #Stack all three RGB 
            img_all = torch.stack((img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:],
                                 img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)
            data = img_all
            data = data.to(device)

            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            #Run necessary train implementations
            output = model(data)
            
            #Zero gradients
            optimizer.zero_grad()

            #forward and backward propagation
            loss_function = nn.MSELoss()
            #Root Mean Squared Error (RMSE) 
            train_loss = torch.sqrt(loss_function(output.float(), target.float()))
            train_loss.backward()
            optimizer.step()

            #Show progress
            if batch_idx % 28 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dataloader.dataset),
                    100. * batch_idx / len(train_dataloader), train_loss.item()))
   
        #To help model improving apply scheduler to reduce learning rate
        scheduler.step()
        curr_lr = scheduler.get_last_lr()
        print("current lr{}".format(curr_lr))     
        
        #Early Stopping
        Curr_Val_loss = validation_23(model)
        print('Valid Epoch: {} Current Validation Loss: {:.6F}'.format(epoch, Curr_Val_loss))
        
        if Curr_Val_loss < 0.242:
            print ("Best model obtained! Set Patience = 2")
            patience = 2 
        
        if Best_Val_loss is None:
            Best_Val_loss = Curr_Val_loss - 0.0001
        #Model didn't improve so add trigger time
        if Curr_Val_loss > Best_Val_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        #Reset to trigger_time 0 and update best validation loss
        else:
            trigger_times = 0
            Best_Val_loss = Curr_Val_loss
            print('trigger times: 0 Best_Val_loss: {:.6F}'.format(Best_Val_loss)) 

    return model

In [9]:
def validation_23(model):
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(validation_dataloader):
            #Stack all three RGB and depth images
            img_all = torch.stack((img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:],
                                 img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)
            data = img_all
            data = data.to(device)
            
            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            output = model(data)
            loss_function = nn.MSELoss()
            loss = torch.sqrt(loss_function(output.float(), target.float()))
            loss_total += loss.item()
    return loss_total / len(validation_dataloader)

In [10]:
model_6 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
model_6.conv1 = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_6.fc = nn.Linear(2048, num_classes)
model_6 = model_6.cuda()
optimizer = torch.optim.Adam(model_6.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
model_6 = train_23(epochs, model_6, optimizer, scheduler)

Train Epoch: 0 [0/2716 (0%)]	Loss: 6.646207
Train Epoch: 0 [896/2716 (33%)]	Loss: 1.067906
Train Epoch: 0 [1792/2716 (66%)]	Loss: 0.975769
Train Epoch: 0 [2352/2716 (99%)]	Loss: 0.929452
current lr[0.0008]
Valid Epoch: 0 Current Validation Loss: 0.842328
trigger times: 1
Train Epoch: 1 [0/2716 (0%)]	Loss: 0.738913
Train Epoch: 1 [896/2716 (33%)]	Loss: 0.556920
Train Epoch: 1 [1792/2716 (66%)]	Loss: 0.538406
Train Epoch: 1 [2352/2716 (99%)]	Loss: 0.573333
current lr[0.00064]
Valid Epoch: 1 Current Validation Loss: 0.610808
trigger times: 0 Best_Val_loss: 0.610808
Train Epoch: 2 [0/2716 (0%)]	Loss: 0.578105
Train Epoch: 2 [896/2716 (33%)]	Loss: 0.513649
Train Epoch: 2 [1792/2716 (66%)]	Loss: 0.442656
Train Epoch: 2 [2352/2716 (99%)]	Loss: 0.450743
current lr[0.0005120000000000001]
Valid Epoch: 2 Current Validation Loss: 0.522986
trigger times: 0 Best_Val_loss: 0.522986
Train Epoch: 3 [0/2716 (0%)]	Loss: 0.465420
Train Epoch: 3 [896/2716 (33%)]	Loss: 0.501209
Train Epoch: 3 [1792/2716 (66

In [11]:
#Save my CNN model
torch.save(model_6.state_dict(), "../model/model_img23.pt")

### RGB + Depth 

In [10]:
def train_1234(epoch, model, optimizer, scheduler):
    # Early stopping Parameters
    Best_Val_loss = None
    patience = 5
    trigger_times = 0
    for epoch in range(0, epochs):
        model.train()
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(train_dataloader): 
            #Stack all three RGB 
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:],
                                 img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:],
                                 img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)
            data = torch.cat((img_all,depth), 1)
            data = data.to(device)

            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            #Run necessary train implementations
            output = model(data)
            
            #Zero gradients
            optimizer.zero_grad()

            #forward and backward propagation
            loss_function = nn.MSELoss()
            #Root Mean Squared Error (RMSE) 
            train_loss = torch.sqrt(loss_function(output.float(), target.float()))
            train_loss.backward()
            optimizer.step()

            #Show progress
            if batch_idx % 28 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dataloader.dataset),
                    100. * batch_idx / len(train_dataloader), train_loss.item()))
   
        #To help model improving apply scheduler to reduce learning rate
        scheduler.step()
        curr_lr = scheduler.get_last_lr()
        print("current lr{}".format(curr_lr))     
        
        #Early Stopping
        Curr_Val_loss = validation_1234(model)
        print('Valid Epoch: {} Current Validation Loss: {:.6F}'.format(epoch, Curr_Val_loss))
        
        if Curr_Val_loss < 0.242:
            print ("Best model obtained! Set Patience = 2")
            patience = 2 
        
        if Best_Val_loss is None:
            Best_Val_loss = Curr_Val_loss - 0.0001
        #Model didn't improve so add trigger time
        if Curr_Val_loss > Best_Val_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        #Reset to trigger_time 0 and update best validation loss
        else:
            trigger_times = 0
            Best_Val_loss = Curr_Val_loss
            print('trigger times: 0 Best_Val_loss: {:.6F}'.format(Best_Val_loss)) 

    return model

In [11]:
def validation_1234(model):
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(validation_dataloader):
            #Stack all three RGB and depth images
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:],
                                 img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:],
                                 img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)
            data = torch.cat((img_all,depth), 1)
            data = data.to(device)
            
            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            output = model(data)
            loss_function = nn.MSELoss()
            loss = torch.sqrt(loss_function(output.float(), target.float()))
            loss_total += loss.item()
    return loss_total / len(validation_dataloader)

In [12]:
model_7 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
model_7.conv1 = nn.Conv2d(12, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_7.fc = nn.Linear(2048, num_classes)
model_7 = model_7.cuda()
optimizer = torch.optim.Adam(model_7.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
model_7 = train_1234(epochs, model_7, optimizer, scheduler)

Train Epoch: 0 [0/2716 (0%)]	Loss: 6.742320
Train Epoch: 0 [896/2716 (33%)]	Loss: 0.906475
Train Epoch: 0 [1792/2716 (66%)]	Loss: 0.874766
Train Epoch: 0 [2352/2716 (99%)]	Loss: 0.623889
current lr[0.0008]
Valid Epoch: 0 Current Validation Loss: 0.707140
trigger times: 1
Train Epoch: 1 [0/2716 (0%)]	Loss: 0.637969
Train Epoch: 1 [896/2716 (33%)]	Loss: 0.554065
Train Epoch: 1 [1792/2716 (66%)]	Loss: 0.456797
Train Epoch: 1 [2352/2716 (99%)]	Loss: 0.600957
current lr[0.00064]
Valid Epoch: 1 Current Validation Loss: 0.495250
trigger times: 0 Best_Val_loss: 0.495250
Train Epoch: 2 [0/2716 (0%)]	Loss: 0.432662
Train Epoch: 2 [896/2716 (33%)]	Loss: 0.476900
Train Epoch: 2 [1792/2716 (66%)]	Loss: 0.491509
Train Epoch: 2 [2352/2716 (99%)]	Loss: 0.465546
current lr[0.0005120000000000001]
Valid Epoch: 2 Current Validation Loss: 0.466839
trigger times: 0 Best_Val_loss: 0.466839
Train Epoch: 3 [0/2716 (0%)]	Loss: 0.526492
Train Epoch: 3 [896/2716 (33%)]	Loss: 0.321943
Train Epoch: 3 [1792/2716 (66

In [13]:
#Save my CNN model
torch.save(model_7.state_dict(), "../model/model_img1234.pt")

### 1&Depth Image

In [14]:
def train_14(epoch, model, optimizer, scheduler):
    # Early stopping Parameters
    Best_Val_loss = None
    patience = 5
    trigger_times = 0
    for epoch in range(0, epochs):
        model.train()
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(train_dataloader): 
            #Stack all three RGB 
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:]),1)
            data = torch.cat((img_all,depth), 1)
            data = data.to(device)

            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            #Run necessary train implementations
            output = model(data)
            
            #Zero gradients
            optimizer.zero_grad()

            #forward and backward propagation
            loss_function = nn.MSELoss()
            #Root Mean Squared Error (RMSE) 
            train_loss = torch.sqrt(loss_function(output.float(), target.float()))
            train_loss.backward()
            optimizer.step()

            #Show progress
            if batch_idx % 28 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dataloader.dataset),
                    100. * batch_idx / len(train_dataloader), train_loss.item()))
   
        #To help model improving apply scheduler to reduce learning rate
        scheduler.step()
        curr_lr = scheduler.get_last_lr()
        print("current lr{}".format(curr_lr))     
        
        #Early Stopping
        Curr_Val_loss = validation_14(model)
        print('Valid Epoch: {} Current Validation Loss: {:.6F}'.format(epoch, Curr_Val_loss))
        
        if Curr_Val_loss < 0.242:
            print ("Best model obtained! Set Patience = 2")
            patience = 2 
        
        if Best_Val_loss is None:
            Best_Val_loss = Curr_Val_loss - 0.0001
        #Model didn't improve so add trigger time
        if Curr_Val_loss > Best_Val_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        #Reset to trigger_time 0 and update best validation loss
        else:
            trigger_times = 0
            Best_Val_loss = Curr_Val_loss
            print('trigger times: 0 Best_Val_loss: {:.6F}'.format(Best_Val_loss)) 

    return model
def validation_14(model):
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(validation_dataloader):
            #Stack all three RGB and depth images
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:]),1)
            data = torch.cat((img_all,depth), 1)
            data = data.to(device)
            
            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            output = model(data)
            loss_function = nn.MSELoss()
            loss = torch.sqrt(loss_function(output.float(), target.float()))
            loss_total += loss.item()
    return loss_total / len(validation_dataloader)


In [15]:
model_8 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
model_8.conv1 = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_8.fc = nn.Linear(2048, num_classes)
model_8 = model_8.cuda()
optimizer = torch.optim.Adam(model_8.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
model_8 = train_14(epochs, model_8, optimizer, scheduler)

Train Epoch: 0 [0/2716 (0%)]	Loss: 6.627444
Train Epoch: 0 [896/2716 (33%)]	Loss: 1.059781
Train Epoch: 0 [1792/2716 (66%)]	Loss: 0.883187
Train Epoch: 0 [2352/2716 (99%)]	Loss: 0.741680
current lr[0.0008]
Valid Epoch: 0 Current Validation Loss: 0.617710
trigger times: 1
Train Epoch: 1 [0/2716 (0%)]	Loss: 0.541774
Train Epoch: 1 [896/2716 (33%)]	Loss: 0.479948
Train Epoch: 1 [1792/2716 (66%)]	Loss: 0.429911
Train Epoch: 1 [2352/2716 (99%)]	Loss: 0.483050
current lr[0.00064]
Valid Epoch: 1 Current Validation Loss: 0.513873
trigger times: 0 Best_Val_loss: 0.513873
Train Epoch: 2 [0/2716 (0%)]	Loss: 0.467984
Train Epoch: 2 [896/2716 (33%)]	Loss: 0.382958
Train Epoch: 2 [1792/2716 (66%)]	Loss: 0.459123
Train Epoch: 2 [2352/2716 (99%)]	Loss: 0.447945
current lr[0.0005120000000000001]
Valid Epoch: 2 Current Validation Loss: 0.472017
trigger times: 0 Best_Val_loss: 0.472017
Train Epoch: 3 [0/2716 (0%)]	Loss: 0.367087
Train Epoch: 3 [896/2716 (33%)]	Loss: 0.280971
Train Epoch: 3 [1792/2716 (66

In [16]:
#Save my CNN model
torch.save(model_8.state_dict(), "../model/model_img14.pt")

### 2&Depth Image

In [8]:
def train_24(epoch, model, optimizer, scheduler):
    # Early stopping Parameters
    Best_Val_loss = None
    patience = 5
    trigger_times = 0
    for epoch in range(0, epochs):
        model.train()
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(train_dataloader): 
            #Stack all three RGB 
            img_all = torch.stack((img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:]),1)
            data = torch.cat((img_all,depth), 1)
            data = data.to(device)

            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            #Run necessary train implementations
            output = model(data)
            
            #Zero gradients
            optimizer.zero_grad()

            #forward and backward propagation
            loss_function = nn.MSELoss()
            #Root Mean Squared Error (RMSE) 
            train_loss = torch.sqrt(loss_function(output.float(), target.float()))
            train_loss.backward()
            optimizer.step()

            #Show progress
            if batch_idx % 28 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dataloader.dataset),
                    100. * batch_idx / len(train_dataloader), train_loss.item()))
   
        #To help model improving apply scheduler to reduce learning rate
        scheduler.step()
        curr_lr = scheduler.get_last_lr()
        print("current lr{}".format(curr_lr))     
        
        #Early Stopping
        Curr_Val_loss = validation_24(model)
        print('Valid Epoch: {} Current Validation Loss: {:.6F}'.format(epoch, Curr_Val_loss))
        
        if Curr_Val_loss < 0.242:
            print ("Best model obtained! Set Patience = 2")
            patience = 2 
        
        if Best_Val_loss is None:
            Best_Val_loss = Curr_Val_loss - 0.0001
        #Model didn't improve so add trigger time
        if Curr_Val_loss > Best_Val_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        #Reset to trigger_time 0 and update best validation loss
        else:
            trigger_times = 0
            Best_Val_loss = Curr_Val_loss
            print('trigger times: 0 Best_Val_loss: {:.6F}'.format(Best_Val_loss)) 

    return model
def validation_24(model):
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(validation_dataloader):
            #Stack all three RGB and depth images
            img_all = torch.stack((img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:]),1)
            data = torch.cat((img_all,depth), 1)
            data = data.to(device)
            
            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            output = model(data)
            loss_function = nn.MSELoss()
            loss = torch.sqrt(loss_function(output.float(), target.float()))
            loss_total += loss.item()
    return loss_total / len(validation_dataloader)


In [9]:
model_9 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
model_9.conv1 = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_9.fc = nn.Linear(2048, num_classes)
model_9 = model_9.cuda()
optimizer = torch.optim.Adam(model_9.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
model_9 = train_24(epochs, model_9, optimizer, scheduler)

Train Epoch: 0 [0/2716 (0%)]	Loss: 6.725142
Train Epoch: 0 [896/2716 (33%)]	Loss: 1.086211
Train Epoch: 0 [1792/2716 (66%)]	Loss: 0.970532
Train Epoch: 0 [2352/2716 (99%)]	Loss: 0.810587
current lr[0.0008]
Valid Epoch: 0 Current Validation Loss: 0.837190
trigger times: 1
Train Epoch: 1 [0/2716 (0%)]	Loss: 0.599671
Train Epoch: 1 [896/2716 (33%)]	Loss: 0.593924
Train Epoch: 1 [1792/2716 (66%)]	Loss: 0.607121
Train Epoch: 1 [2352/2716 (99%)]	Loss: 0.567571
current lr[0.00064]
Valid Epoch: 1 Current Validation Loss: 0.748171
trigger times: 0 Best_Val_loss: 0.748171
Train Epoch: 2 [0/2716 (0%)]	Loss: 0.607846
Train Epoch: 2 [896/2716 (33%)]	Loss: 0.507660
Train Epoch: 2 [1792/2716 (66%)]	Loss: 0.475345
Train Epoch: 2 [2352/2716 (99%)]	Loss: 0.688361
current lr[0.0005120000000000001]
Valid Epoch: 2 Current Validation Loss: 0.606348
trigger times: 0 Best_Val_loss: 0.606348
Train Epoch: 3 [0/2716 (0%)]	Loss: 0.507427
Train Epoch: 3 [896/2716 (33%)]	Loss: 0.528174
Train Epoch: 3 [1792/2716 (66

In [10]:
#Save my CNN model
torch.save(model_9.state_dict(), "../model/model_img24.pt")

### 3&Depth Image

In [11]:
def train_34(epoch, model, optimizer, scheduler):
    # Early stopping Parameters
    Best_Val_loss = None
    patience = 5
    trigger_times = 0
    for epoch in range(0, epochs):
        model.train()
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(train_dataloader): 
            #Stack all three RGB 
            img_all = torch.stack((img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)
            data = torch.cat((img_all,depth), 1)
            data = data.to(device)

            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            #Run necessary train implementations
            output = model(data)
            
            #Zero gradients
            optimizer.zero_grad()

            #forward and backward propagation
            loss_function = nn.MSELoss()
            #Root Mean Squared Error (RMSE) 
            train_loss = torch.sqrt(loss_function(output.float(), target.float()))
            train_loss.backward()
            optimizer.step()

            #Show progress
            if batch_idx % 28 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dataloader.dataset),
                    100. * batch_idx / len(train_dataloader), train_loss.item()))
   
        #To help model improving apply scheduler to reduce learning rate
        scheduler.step()
        curr_lr = scheduler.get_last_lr()
        print("current lr{}".format(curr_lr))     
        
        #Early Stopping
        Curr_Val_loss = validation_34(model)
        print('Valid Epoch: {} Current Validation Loss: {:.6F}'.format(epoch, Curr_Val_loss))
        
        if Curr_Val_loss < 0.242:
            print ("Best model obtained! Set Patience = 2")
            patience = 2 
        
        if Best_Val_loss is None:
            Best_Val_loss = Curr_Val_loss - 0.0001
        #Model didn't improve so add trigger time
        if Curr_Val_loss > Best_Val_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        #Reset to trigger_time 0 and update best validation loss
        else:
            trigger_times = 0
            Best_Val_loss = Curr_Val_loss
            print('trigger times: 0 Best_Val_loss: {:.6F}'.format(Best_Val_loss)) 

    return model
def validation_34(model):
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(validation_dataloader):
            #Stack all three RGB and depth images
            img_all = torch.stack((img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)
            data = torch.cat((img_all,depth), 1)
            data = data.to(device)
            
            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            output = model(data)
            loss_function = nn.MSELoss()
            loss = torch.sqrt(loss_function(output.float(), target.float()))
            loss_total += loss.item()
    return loss_total / len(validation_dataloader)


In [12]:
model_10 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
model_10.conv1 = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_10.fc = nn.Linear(2048, num_classes)
model_10 = model_10.cuda()
optimizer = torch.optim.Adam(model_10.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
model_10 = train_34(epochs, model_10, optimizer, scheduler)

Train Epoch: 0 [0/2716 (0%)]	Loss: 6.718586
Train Epoch: 0 [896/2716 (33%)]	Loss: 1.256243
Train Epoch: 0 [1792/2716 (66%)]	Loss: 0.867405
Train Epoch: 0 [2352/2716 (99%)]	Loss: 0.680487
current lr[0.0008]
Valid Epoch: 0 Current Validation Loss: 0.677068
trigger times: 1
Train Epoch: 1 [0/2716 (0%)]	Loss: 0.611126
Train Epoch: 1 [896/2716 (33%)]	Loss: 0.611261
Train Epoch: 1 [1792/2716 (66%)]	Loss: 0.830169
Train Epoch: 1 [2352/2716 (99%)]	Loss: 0.588553
current lr[0.00064]
Valid Epoch: 1 Current Validation Loss: 1.144233
trigger times: 2
Train Epoch: 2 [0/2716 (0%)]	Loss: 0.813097
Train Epoch: 2 [896/2716 (33%)]	Loss: 0.573353
Train Epoch: 2 [1792/2716 (66%)]	Loss: 0.354229
Train Epoch: 2 [2352/2716 (99%)]	Loss: 0.499411
current lr[0.0005120000000000001]
Valid Epoch: 2 Current Validation Loss: 0.527793
trigger times: 0 Best_Val_loss: 0.527793
Train Epoch: 3 [0/2716 (0%)]	Loss: 0.394195
Train Epoch: 3 [896/2716 (33%)]	Loss: 0.366397
Train Epoch: 3 [1792/2716 (66%)]	Loss: 0.435957
Train

In [13]:
#Save my CNN model
torch.save(model_10.state_dict(), "../model/model_img34.pt")

### RGB Images

In [8]:
def train_123(epoch, model, optimizer, scheduler):
    # Early stopping Parameters
    Best_Val_loss = None
    patience = 5
    trigger_times = 0
    for epoch in range(0, epochs):
        model.train()
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(train_dataloader): 
            #Stack all three RGB 
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:],
                                  img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:],
                                  img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)
            data = img_all
            data = data.to(device)

            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            #Run necessary train implementations
            output = model(data)
            
            #Zero gradients
            optimizer.zero_grad()

            #forward and backward propagation
            loss_function = nn.MSELoss()
            #Root Mean Squared Error (RMSE) 
            train_loss = torch.sqrt(loss_function(output.float(), target.float()))
            train_loss.backward()
            optimizer.step()

            #Show progress
            if batch_idx % 28 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dataloader.dataset),
                    100. * batch_idx / len(train_dataloader), train_loss.item()))
   
        #To help model improving apply scheduler to reduce learning rate
        scheduler.step()
        curr_lr = scheduler.get_last_lr()
        print("current lr{}".format(curr_lr))     
        
        #Early Stopping
        Curr_Val_loss = validation_123(model)
        print('Valid Epoch: {} Current Validation Loss: {:.6F}'.format(epoch, Curr_Val_loss))
        
        if Curr_Val_loss < 0.242:
            print ("Best model obtained! Set Patience = 2")
            patience = 2 
        
        if Best_Val_loss is None:
            Best_Val_loss = Curr_Val_loss - 0.0001
        #Model didn't improve so add trigger time
        if Curr_Val_loss > Best_Val_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        #Reset to trigger_time 0 and update best validation loss
        else:
            trigger_times = 0
            Best_Val_loss = Curr_Val_loss
            print('trigger times: 0 Best_Val_loss: {:.6F}'.format(Best_Val_loss)) 

    return model
def validation_123(model):
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(validation_dataloader):
            #Stack all three RGB and depth images
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:],
                                  img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:],
                                  img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)
            data = img_all
            data = data.to(device)
            
            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            output = model(data)
            loss_function = nn.MSELoss()
            loss = torch.sqrt(loss_function(output.float(), target.float()))
            loss_total += loss.item()
    return loss_total / len(validation_dataloader)


In [10]:
model_11 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
model_11.conv1 = nn.Conv2d(9, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_11.fc = nn.Linear(2048, num_classes)
model_11 = model_11.cuda()
optimizer = torch.optim.Adam(model_11.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
model_11 = train_123(epochs, model_11, optimizer, scheduler)

Train Epoch: 0 [0/2716 (0%)]	Loss: 6.809105
Train Epoch: 0 [896/2716 (33%)]	Loss: 1.408636
Train Epoch: 0 [1792/2716 (66%)]	Loss: 0.774298
Train Epoch: 0 [2352/2716 (99%)]	Loss: 0.746768
current lr[0.0008]
Valid Epoch: 0 Current Validation Loss: 0.764998
trigger times: 1
Train Epoch: 1 [0/2716 (0%)]	Loss: 0.770220
Train Epoch: 1 [896/2716 (33%)]	Loss: 0.570433
Train Epoch: 1 [1792/2716 (66%)]	Loss: 0.457122
Train Epoch: 1 [2352/2716 (99%)]	Loss: 0.454554
current lr[0.00064]
Valid Epoch: 1 Current Validation Loss: 0.493813
trigger times: 0 Best_Val_loss: 0.493813
Train Epoch: 2 [0/2716 (0%)]	Loss: 0.461336
Train Epoch: 2 [896/2716 (33%)]	Loss: 0.412427
Train Epoch: 2 [1792/2716 (66%)]	Loss: 0.473053
Train Epoch: 2 [2352/2716 (99%)]	Loss: 0.466055
current lr[0.0005120000000000001]
Valid Epoch: 2 Current Validation Loss: 0.457316
trigger times: 0 Best_Val_loss: 0.457316
Train Epoch: 3 [0/2716 (0%)]	Loss: 0.373526
Train Epoch: 3 [896/2716 (33%)]	Loss: 0.413913
Train Epoch: 3 [1792/2716 (66

In [11]:
#Save my CNN model
torch.save(model_11.state_dict(), "../model/model_img123.pt")

### Depth Image

In [13]:
def train_4(epoch, model, optimizer, scheduler):
    # Early stopping Parameters
    Best_Val_loss = None
    patience = 5
    trigger_times = 0
    for epoch in range(0, epochs):
        model.train()
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(train_dataloader): 
            #Stack all three RGB 
            data = depth
            data = data.to(device)

            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            #Run necessary train implementations
            output = model(data)
            
            #Zero gradients
            optimizer.zero_grad()

            #forward and backward propagation
            loss_function = nn.MSELoss()
            #Root Mean Squared Error (RMSE) 
            train_loss = torch.sqrt(loss_function(output.float(), target.float()))
            train_loss.backward()
            optimizer.step()

            #Show progress
            if batch_idx % 28 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dataloader.dataset),
                    100. * batch_idx / len(train_dataloader), train_loss.item()))
   
        #To help model improving apply scheduler to reduce learning rate
        scheduler.step()
        curr_lr = scheduler.get_last_lr()
        print("current lr{}".format(curr_lr))     
        
        #Early Stopping
        Curr_Val_loss = validation_4(model)
        print('Valid Epoch: {} Current Validation Loss: {:.6F}'.format(epoch, Curr_Val_loss))
        
        if Curr_Val_loss < 0.242:
            print ("Best model obtained! Set Patience = 2")
            patience = 2 
        
        if Best_Val_loss is None:
            Best_Val_loss = Curr_Val_loss - 0.0001
        #Model didn't improve so add trigger time
        if Curr_Val_loss > Best_Val_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        #Reset to trigger_time 0 and update best validation loss
        else:
            trigger_times = 0
            Best_Val_loss = Curr_Val_loss
            print('trigger times: 0 Best_Val_loss: {:.6F}'.format(Best_Val_loss)) 

    return model
def validation_4(model):
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(validation_dataloader):
            data = depth
            data = data.to(device)
            
            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            output = model(data)
            loss_function = nn.MSELoss()
            loss = torch.sqrt(loss_function(output.float(), target.float()))
            loss_total += loss.item()
    return loss_total / len(validation_dataloader)

In [14]:
model_12 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
model_12.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_12.fc = nn.Linear(2048, num_classes)
model_12 = model_12.cuda()
optimizer = torch.optim.Adam(model_12.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
model_12 = train_4(epochs, model_12, optimizer, scheduler)

Train Epoch: 0 [0/2716 (0%)]	Loss: 6.673417
Train Epoch: 0 [896/2716 (33%)]	Loss: 1.550668
Train Epoch: 0 [1792/2716 (66%)]	Loss: 0.923565
Train Epoch: 0 [2352/2716 (99%)]	Loss: 1.056602
current lr[0.0008]
Valid Epoch: 0 Current Validation Loss: 1.074201
trigger times: 1
Train Epoch: 1 [0/2716 (0%)]	Loss: 0.748698
Train Epoch: 1 [896/2716 (33%)]	Loss: 0.702109
Train Epoch: 1 [1792/2716 (66%)]	Loss: 0.672973
Train Epoch: 1 [2352/2716 (99%)]	Loss: 0.522197
current lr[0.00064]
Valid Epoch: 1 Current Validation Loss: 0.582641
trigger times: 0 Best_Val_loss: 0.582641
Train Epoch: 2 [0/2716 (0%)]	Loss: 0.482117
Train Epoch: 2 [896/2716 (33%)]	Loss: 0.465239
Train Epoch: 2 [1792/2716 (66%)]	Loss: 0.554848
Train Epoch: 2 [2352/2716 (99%)]	Loss: 0.434390
current lr[0.0005120000000000001]
Valid Epoch: 2 Current Validation Loss: 0.434109
trigger times: 0 Best_Val_loss: 0.434109
Train Epoch: 3 [0/2716 (0%)]	Loss: 0.394535
Train Epoch: 3 [896/2716 (33%)]	Loss: 0.399299
Train Epoch: 3 [1792/2716 (66

In [15]:
#Save my CNN model
torch.save(model_12.state_dict(), "../model/model_img4.pt")

### First Second Depth image

In [8]:
def train_124(epoch, model, optimizer, scheduler):
    # Early stopping Parameters
    Best_Val_loss = None
    patience = 5
    trigger_times = 0
    for epoch in range(0, epochs):
        model.train()
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(train_dataloader): 
            #Stack all three RGB 
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:],
                                   img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:]),1)
            data = torch.cat((img_all,depth), 1)
            data = data.to(device)

            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            #Run necessary train implementations
            output = model(data)
            
            #Zero gradients
            optimizer.zero_grad()

            #forward and backward propagation
            loss_function = nn.MSELoss()
            #Root Mean Squared Error (RMSE) 
            train_loss = torch.sqrt(loss_function(output.float(), target.float()))
            train_loss.backward()
            optimizer.step()

            #Show progress
            if batch_idx % 28 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dataloader.dataset),
                    100. * batch_idx / len(train_dataloader), train_loss.item()))
   
        #To help model improving apply scheduler to reduce learning rate
        scheduler.step()
        curr_lr = scheduler.get_last_lr()
        print("current lr{}".format(curr_lr))     
        
        #Early Stopping
        Curr_Val_loss = validation_124(model)
        print('Valid Epoch: {} Current Validation Loss: {:.6F}'.format(epoch, Curr_Val_loss))
        
        if Curr_Val_loss < 0.242:
            print ("Best model obtained! Set Patience = 2")
            patience = 2 
        
        if Best_Val_loss is None:
            Best_Val_loss = Curr_Val_loss - 0.0001
        #Model didn't improve so add trigger time
        if Curr_Val_loss > Best_Val_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        #Reset to trigger_time 0 and update best validation loss
        else:
            trigger_times = 0
            Best_Val_loss = Curr_Val_loss
            print('trigger times: 0 Best_Val_loss: {:.6F}'.format(Best_Val_loss)) 

    return model
def validation_124(model):
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(validation_dataloader):
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:],
                                   img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:]),1)
            data = torch.cat((img_all,depth), 1)
            data = data.to(device)
            
            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            output = model(data)
            loss_function = nn.MSELoss()
            loss = torch.sqrt(loss_function(output.float(), target.float()))
            loss_total += loss.item()
    return loss_total / len(validation_dataloader)

In [9]:
model_13 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
model_13.conv1 = nn.Conv2d(9, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_13.fc = nn.Linear(2048, num_classes)
model_13 = model_13.cuda()
optimizer = torch.optim.Adam(model_13.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
model_13 = train_124(epochs, model_13, optimizer, scheduler)

Train Epoch: 0 [0/2716 (0%)]	Loss: 6.675582
Train Epoch: 0 [896/2716 (33%)]	Loss: 1.264700
Train Epoch: 0 [1792/2716 (66%)]	Loss: 0.991149
Train Epoch: 0 [2352/2716 (99%)]	Loss: 0.655316
current lr[0.0008]
Valid Epoch: 0 Current Validation Loss: 0.833157
trigger times: 1
Train Epoch: 1 [0/2716 (0%)]	Loss: 0.730469
Train Epoch: 1 [896/2716 (33%)]	Loss: 0.574205
Train Epoch: 1 [1792/2716 (66%)]	Loss: 0.532855
Train Epoch: 1 [2352/2716 (99%)]	Loss: 0.433514
current lr[0.00064]
Valid Epoch: 1 Current Validation Loss: 0.548026
trigger times: 0 Best_Val_loss: 0.548026
Train Epoch: 2 [0/2716 (0%)]	Loss: 0.570023
Train Epoch: 2 [896/2716 (33%)]	Loss: 0.497394
Train Epoch: 2 [1792/2716 (66%)]	Loss: 0.393107
Train Epoch: 2 [2352/2716 (99%)]	Loss: 0.484221
current lr[0.0005120000000000001]
Valid Epoch: 2 Current Validation Loss: 0.444670
trigger times: 0 Best_Val_loss: 0.444670
Train Epoch: 3 [0/2716 (0%)]	Loss: 0.474951
Train Epoch: 3 [896/2716 (33%)]	Loss: 0.431100
Train Epoch: 3 [1792/2716 (66

In [10]:
#Save my CNN model
torch.save(model_13.state_dict(), "../model/model_img124.pt")

### First Third Depth

In [11]:
def train_134(epoch, model, optimizer, scheduler):
    # Early stopping Parameters
    Best_Val_loss = None
    patience = 5
    trigger_times = 0
    for epoch in range(0, epochs):
        model.train()
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(train_dataloader): 
            #Stack all three RGB 
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:],
                                   img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)
            data = torch.cat((img_all,depth), 1)
            data = data.to(device)

            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            #Run necessary train implementations
            output = model(data)
            
            #Zero gradients
            optimizer.zero_grad()

            #forward and backward propagation
            loss_function = nn.MSELoss()
            #Root Mean Squared Error (RMSE) 
            train_loss = torch.sqrt(loss_function(output.float(), target.float()))
            train_loss.backward()
            optimizer.step()

            #Show progress
            if batch_idx % 28 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dataloader.dataset),
                    100. * batch_idx / len(train_dataloader), train_loss.item()))
   
        #To help model improving apply scheduler to reduce learning rate
        scheduler.step()
        curr_lr = scheduler.get_last_lr()
        print("current lr{}".format(curr_lr))     
        
        #Early Stopping
        Curr_Val_loss = validation_134(model)
        print('Valid Epoch: {} Current Validation Loss: {:.6F}'.format(epoch, Curr_Val_loss))
        
        if Curr_Val_loss < 0.242:
            print ("Best model obtained! Set Patience = 2")
            patience = 2 
        
        if Best_Val_loss is None:
            Best_Val_loss = Curr_Val_loss - 0.0001
        #Model didn't improve so add trigger time
        if Curr_Val_loss > Best_Val_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        #Reset to trigger_time 0 and update best validation loss
        else:
            trigger_times = 0
            Best_Val_loss = Curr_Val_loss
            print('trigger times: 0 Best_Val_loss: {:.6F}'.format(Best_Val_loss)) 

    return model
def validation_134(model):
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(validation_dataloader):
            img_all = torch.stack((img0[:,0,:,:],img0[:,1,:,:],img0[:,2,:,:],
                                   img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)
            data = torch.cat((img_all,depth), 1)
            data = data.to(device)
            
            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            output = model(data)
            loss_function = nn.MSELoss()
            loss = torch.sqrt(loss_function(output.float(), target.float()))
            loss_total += loss.item()
    return loss_total / len(validation_dataloader)

In [12]:
model_14 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
model_14.conv1 = nn.Conv2d(9, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_14.fc = nn.Linear(2048, num_classes)
model_14 = model_14.cuda()
optimizer = torch.optim.Adam(model_14.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
model_14 = train_134(epochs, model_14, optimizer, scheduler)

Train Epoch: 0 [0/2716 (0%)]	Loss: 6.730664
Train Epoch: 0 [896/2716 (33%)]	Loss: 1.074315
Train Epoch: 0 [1792/2716 (66%)]	Loss: 0.749331
Train Epoch: 0 [2352/2716 (99%)]	Loss: 0.687517
current lr[0.0008]
Valid Epoch: 0 Current Validation Loss: 0.604030
trigger times: 1
Train Epoch: 1 [0/2716 (0%)]	Loss: 0.793608
Train Epoch: 1 [896/2716 (33%)]	Loss: 0.456049
Train Epoch: 1 [1792/2716 (66%)]	Loss: 0.527255
Train Epoch: 1 [2352/2716 (99%)]	Loss: 0.538064
current lr[0.00064]
Valid Epoch: 1 Current Validation Loss: 0.486446
trigger times: 0 Best_Val_loss: 0.486446
Train Epoch: 2 [0/2716 (0%)]	Loss: 0.369355
Train Epoch: 2 [896/2716 (33%)]	Loss: 0.446856
Train Epoch: 2 [1792/2716 (66%)]	Loss: 0.370587
Train Epoch: 2 [2352/2716 (99%)]	Loss: 0.339627
current lr[0.0005120000000000001]
Valid Epoch: 2 Current Validation Loss: 0.403123
trigger times: 0 Best_Val_loss: 0.403123
Train Epoch: 3 [0/2716 (0%)]	Loss: 0.362686
Train Epoch: 3 [896/2716 (33%)]	Loss: 0.360950
Train Epoch: 3 [1792/2716 (66

In [13]:
#Save my CNN model
torch.save(model_14.state_dict(), "../model/model_img134.pt")

### Second&Third&Depth Images

In [8]:
def train_234(epoch, model, optimizer, scheduler):
    # Early stopping Parameters
    Best_Val_loss = None
    patience = 5
    trigger_times = 0
    for epoch in range(0, epochs):
        model.train()
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(train_dataloader): 
            #Stack all three RGB 
            img_all = torch.stack((img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:],
                                   img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)
            data = torch.cat((img_all,depth), 1)
            data = data.to(device)

            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            #Run necessary train implementations
            output = model(data)
            
            #Zero gradients
            optimizer.zero_grad()

            #forward and backward propagation
            loss_function = nn.MSELoss()
            #Root Mean Squared Error (RMSE) 
            train_loss = torch.sqrt(loss_function(output.float(), target.float()))
            train_loss.backward()
            optimizer.step()

            #Show progress
            if batch_idx % 28 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dataloader.dataset),
                    100. * batch_idx / len(train_dataloader), train_loss.item()))
   
        #To help model improving apply scheduler to reduce learning rate
        scheduler.step()
        curr_lr = scheduler.get_last_lr()
        print("current lr{}".format(curr_lr))     
        
        #Early Stopping
        Curr_Val_loss = validation_234(model)
        print('Valid Epoch: {} Current Validation Loss: {:.6F}'.format(epoch, Curr_Val_loss))
        
        if Curr_Val_loss < 0.242:
            print ("Best model obtained! Set Patience = 2")
            patience = 2 
        
        if Best_Val_loss is None:
            Best_Val_loss = Curr_Val_loss - 0.0001
        #Model didn't improve so add trigger time
        if Curr_Val_loss > Best_Val_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        #Reset to trigger_time 0 and update best validation loss
        else:
            trigger_times = 0
            Best_Val_loss = Curr_Val_loss
            print('trigger times: 0 Best_Val_loss: {:.6F}'.format(Best_Val_loss)) 

    return model

def validation_234(model):
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for batch_idx, ((img0, img1, img2, depth, field_id), target) in enumerate(validation_dataloader):
            img_all = torch.stack((img1[:,0,:,:],img1[:,1,:,:],img1[:,2,:,:],
                                   img2[:,0,:,:],img2[:,1,:,:],img2[:,2,:,:]),1)
            data = torch.cat((img_all,depth), 1)
            data = data.to(device)
            
            #Send to Device
            target = target.to(device)
            
            #multiply GT by 100
            target = target*100
            
            output = model(data)
            loss_function = nn.MSELoss()
            loss = torch.sqrt(loss_function(output.float(), target.float()))
            loss_total += loss.item()
    return loss_total / len(validation_dataloader)

In [9]:
model_15 = models.resnet152(weights=ResNet152_Weights.DEFAULT)
model_15.conv1 = nn.Conv2d(9, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_15.fc = nn.Linear(2048, num_classes)
model_15 = model_15.cuda()
optimizer = torch.optim.Adam(model_15.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
model_15 = train_234(epochs, model_15, optimizer, scheduler)

Train Epoch: 0 [0/2716 (0%)]	Loss: 6.557008
Train Epoch: 0 [896/2716 (33%)]	Loss: 1.072155
Train Epoch: 0 [1792/2716 (66%)]	Loss: 1.004401
Train Epoch: 0 [2352/2716 (99%)]	Loss: 0.670389
current lr[0.0008]
Valid Epoch: 0 Current Validation Loss: 0.778406
trigger times: 1
Train Epoch: 1 [0/2716 (0%)]	Loss: 0.646696
Train Epoch: 1 [896/2716 (33%)]	Loss: 0.651876
Train Epoch: 1 [1792/2716 (66%)]	Loss: 0.583474
Train Epoch: 1 [2352/2716 (99%)]	Loss: 0.531857
current lr[0.00064]
Valid Epoch: 1 Current Validation Loss: 0.593636
trigger times: 0 Best_Val_loss: 0.593636
Train Epoch: 2 [0/2716 (0%)]	Loss: 0.500524
Train Epoch: 2 [896/2716 (33%)]	Loss: 0.544647
Train Epoch: 2 [1792/2716 (66%)]	Loss: 0.428389
Train Epoch: 2 [2352/2716 (99%)]	Loss: 0.572793
current lr[0.0005120000000000001]
Valid Epoch: 2 Current Validation Loss: 0.571784
trigger times: 0 Best_Val_loss: 0.571784
Train Epoch: 3 [0/2716 (0%)]	Loss: 0.437641
Train Epoch: 3 [896/2716 (33%)]	Loss: 0.394140
Train Epoch: 3 [1792/2716 (66

In [10]:
#Save my CNN model
torch.save(model_15.state_dict(), "../model/model_img234.pt")